In [1]:
import time
import pandas as pd
import numpy as np
import math

In [2]:
current_inventory = {'1': 300,
                    '2': 300,
                    '3': 100,
                    '4': 100,
                    '5': 50,
                    '6': 250,
                    '7': 10,
                    '8': 0,
                    '9': 1}

In [3]:
initial_inventory = current_inventory.copy()

In [4]:
def reset_inventory():
    return initial_inventory.copy()

In [5]:
num_stores = 9
current_day = 0

In [6]:
daily_demands = {'1': [(83,5.8), (15,7), (20,4), (25,6), (10,6), (83,5.8), (20,4)],
                '2': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '3': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '4': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '5': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '6': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)],
                '7': [(20,5), (15,7), (20,4), (25,6), (10,6), (8,5), (20,4)],
                '8': [(20,5), (15,7), (20,4), (25,6), (10,6), (6,2), (20,4)],
                '9': [(20,5), (15,7), (20,4), (25,6), (10,6), (10,7), (20,4)]}


In [7]:
costs = {'stockout': 30,
        'holding': 1,
        'transshipment': 1}

In [8]:
def convolution(store_id, start_day):
    
    store_data = daily_demands[store_id]
    future_data = store_data[start_day:]
    mean_values = [mean for mean, _ in future_data]
    std_dev_values = [std_dev for _, std_dev in future_data]
    convoluted_mean = sum(mean_values)
    convoluted_st_dev = math.sqrt(sum(x**2 for x in std_dev_values))
    return convoluted_mean, convoluted_st_dev

In [9]:
convolution('1', current_day)

(256, 14.84183277092152)

In [10]:
p_value = (costs['stockout'] - costs['transshipment']) / (costs['stockout'] + costs['holding'] - costs['transshipment'])
ratio=norm.ppf(p_value)

0.9666666666666667

In [11]:
current_day = 0
sales_dff = pd.DataFrame(columns=['Store', 'Day', 'Sales','Lost Sales','Demand','Inventory'])

In [12]:
def update_inventory(current_day):
    global sales_dff
    for i in current_inventory.keys():
        
        todays_sales = {i: max(0,int(np.random.normal(daily_demands[(i)][current_day][0], daily_demands[(i)][current_day][1])))}
        actual_sales = min(todays_sales[i], current_inventory[i])
        lost_sales = max(0, todays_sales[i] - current_inventory[i])
        current_inventory[i] = current_inventory[i]-actual_sales
        new_row = pd.DataFrame({'Store': [i], 'Day': [current_day], 'Sales': [actual_sales],'Lost Sales':lost_sales,'Demand':todays_sales[i],'Inventory':current_inventory[i]})
        sales_dff = pd.concat([sales_dff, new_row], ignore_index=True)

    return current_inventory   

In [13]:
current_time=time.time()

results = pd.DataFrame()
cost_dict={}
for week in range(10):
    current_inventory = reset_inventory()
    sales_dff = pd.DataFrame()

    for day in range(7):
        daily_inventory=update_inventory(day)

    weekly_cost=(sales_dff["Lost Sales"].sum()*costs["stockout"])+(sales_dff["Inventory"].sum()*costs["holding"])   
    cost_dict[week+1]=weekly_cost
    sales_dff["Week"]=week+1
    sales_dff["Cost"]=weekly_cost
    results = pd.concat([results,sales_dff], ignore_index=True)
print(cost_dict)    
print(results)


        
        
        
        
        
            
                       
                
               

{1: 17894, 2: 16980, 3: 17186, 4: 16946, 5: 17130, 6: 18052, 7: 18470, 8: 16493, 9: 18373, 10: 17876}
    Store  Day  Sales  Lost Sales  Demand  Inventory  Week   Cost
0       1    0     84           0      84        216     1  17894
1       2    0     21           0      21        279     1  17894
2       3    0     21           0      21         79     1  17894
3       4    0     16           0      16         84     1  17894
4       5    0     16           0      16         34     1  17894
..    ...  ...    ...         ...     ...        ...   ...    ...
625     5    6      0          16      16          0    10  17876
626     6    6     22           0      22        126    10  17876
627     7    6      0          21      21          0    10  17876
628     8    6      0          19      19          0    10  17876
629     9    6      0          22      22          0    10  17876

[630 rows x 8 columns]
